In [1]:
from torchrl.modules import TruncatedNormal
# from timm.layers.weight_init import trunc_normal_
import torch 

x = torch.randn(32, 3, 256, 256)

truncate_normal = TruncatedNormal(x,
                                    scale=0.02)
print(truncate_normal)

/home/manish/anaconda3/envs/cuda121/lib/python3.10/site-packages/torchrl/data/replay_buffers/samplers.py:36: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)


TypeError: expand_as(): argument 'other' (position 1) must be Tensor, not float

# dummy time_embedding 

In [ ]:
import torch 


# (batch_size, tembedding_channels)
temb = torch.randn(2, 512)
temb

tensor([[-1.0657, -0.2787, -1.2110,  ...,  0.5981, -0.5768, -0.5393],
        [ 1.1917, -0.1065,  1.9694,  ..., -0.1435, -1.1355, -0.3291]])

In [1]:
(1,) * 6

(1, 1, 1, 1, 1, 1)

# AdaGroupNorm

`for - images`

In [17]:
from diffusers.models.normalization import AdaGroupNorm
import torch 

# in_channels = 3
out_channels = 6
temb_channels = 6

x = torch.randn(2, out_channels,  64, 64)
temb = torch.randn(2, temb_channels)




ada_group_normalization = AdaGroupNorm(embedding_dim=temb_channels,   # make sure in `embedding_dim` to put the `time_embedding` that is 256 in here.
                                       out_dim=out_channels,
                                       num_groups=2,
                                       eps=1e-6)

print(ada_group_normalization)

ada_group_normalization = ada_group_normalization(x, temb)
ada_group_normalization.shape

AdaGroupNorm(
  (linear): Linear(in_features=6, out_features=12, bias=True)
)


torch.Size([2, 6, 64, 64])

`for - video`

In [2]:
from diffusers.models.normalization import AdaGroupNorm
import torch 

in_channels = 3
out_channels = 64
temb_channels = 8
batch_size = 2
frame = 8 

batch_frame = batch_size * frame

# video_x = torch.randn(batch_size, out_channels, frame,  64, 64)
image_x = torch.randn(batch_frame, out_channels, 64, 64) # so the total image = 2 * 8 => 16 images
temb = torch.randn(batch_frame, temb_channels)




ada_group_normalization = AdaGroupNorm(embedding_dim=temb_channels,   # make sure in `embedding_dim` to put the `time_embedding` that is 256 in here.
                                       out_dim=out_channels,
                                       num_groups=batch_size,
                                       eps=1e-6)

print(ada_group_normalization)

ada_group_normalization = ada_group_normalization(image_x, temb)
ada_group_normalization.shape

AdaGroupNorm(
  (linear): Linear(in_features=8, out_features=128, bias=True)
)
what is the shape of row data: torch.Size([16, 64, 64, 64]) and what is the shape of scale: torch.Size([16, 64, 1, 1]) and what is the shape of shift: torch.Size([16, 64, 1, 1])


torch.Size([16, 64, 64, 64])

# SpatialNorm

In [33]:
import torch 
from torch import nn 
from torch.nn import functional as F 

class SpatialNorm(nn.Module):
    """
    Spatially conditioned normalization as defined in https://arxiv.org/abs/2209.09002.

    Args:
        f_channels (`int`):
            The number of channels for input to group normalization layer, and output of the spatial norm layer.
        zq_channels (`int`):
            The number of channels for the quantized vector as described in the paper.
    """

    def __init__(
        self,
        f_channels: int,
        zq_channels: int,
    ):
        super().__init__()
        self.norm_layer = nn.GroupNorm(num_channels=f_channels, num_groups=32, eps=1e-6, affine=True)
        self.conv_y = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)
        self.conv_b = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, f: torch.Tensor, zq: torch.Tensor) -> torch.Tensor:
        print(f"what is the shape of f : {f.shape} and zq: {zq.shape}")

        f_size = f.shape[-2:] # (height, width)
        # print(f"so if i add the new dim: {f_size, None}")

        zq = F.interpolate(zq, size=f_size, mode="nearest")
        norm_f = self.norm_layer(f)
        new_f = norm_f * self.conv_y(zq) + self.conv_b(zq)
        return new_f

In [34]:



num_channels = 64
zq_channels = 8

spatial_norm = SpatialNorm(f_channels=num_channels,
                           zq_channels=zq_channels)

print(spatial_norm)

x = torch.randn(2, 3, 64, 64)
temb = torch.randn(2, 128)

spatial_norm = spatial_norm(x, temb)
spatial_norm





SpatialNorm(
  (norm_layer): GroupNorm(32, 64, eps=1e-06, affine=True)
  (conv_y): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv_b): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
)
what is the shape of f : torch.Size([2, 3, 64, 64]) and zq: torch.Size([2, 128])


ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [] and output size of torch.Size([64, 64]). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.

# torch.interpolate()

#### `mode: 'nearest'`

In [16]:
import torch
import torch.nn.functional as F

# x = torch.rand(1, 1, 2, 2)
x = torch.rand(32, 3, 256, 256)

nearest_interpolate = F.interpolate(x, 
                                #  size=(64, 64),
                                scale_factor=(4, 4),
                                 mode='nearest')
nearest_interpolate.shape


torch.Size([32, 3, 1024, 1024])

#### `mode: 'Linear'`

In [39]:
# batch_size = 32, channels=3, length=4
x_linear = torch.randn(32, 3, 4)




linear_interpolate = F.interpolate(x_linear,
                                   size=8,
                                   mode='linear',
                                   align_corners=False  # a/c to pytorch library
                                   )

linear_interpolate.shape



torch.Size([32, 3, 8])

#### `mode: 'BiLinear'`

In [43]:

bilinear_interpolate = F.interpolate(x,
                                     size=(64, 64),
                                     mode='bilinear')
bilinear_interpolate.shape

torch.Size([32, 3, 64, 64])

In [3]:
import torch 
from torch import nn 
from typing import Optional, Tuple, Union

class Neural_Network(nn.Module):

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Union[int, Tuple[int, int, int]] = 3,
                 stride: Union[int, Tuple[int, int, int]] = 1,
                 group: Optional[int] = 32,
                 dilation: Optional[int] = 1,
                 padding: Optional[int] = 1):
        
        super().__init__()
        
        self.norm1 = nn.GroupNorm(num_groups=group,
                             num_channels=in_channels,
                             eps=1e-6)
        
        self.conv1 = nn.Conv3d(in_channels=in_channels,
                          out_channels=out_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding,
                          dilation=dilation)
        


    def forward(self, x):

        x = self.norm1(x)
        x = self.conv1(x)
        return x 
    

        


x = torch.randn(2, 16, 8, 256, 256)

neural_network = Neural_Network(in_channels=16,
                                out_channels=16,
                                group=2)

print(neural_network)

output = neural_network(x)
output.shape


Neural_Network(
  (norm1): GroupNorm(2, 16, eps=1e-06, affine=True)
  (conv1): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
)


torch.Size([2, 16, 8, 256, 256])